<a href="https://colab.research.google.com/github/hhamaguchi/BIG_DATA/blob/master/Projeto_DMSN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Instalação de bibliotecas
!pip3 install -q numpy scipy pandas twython
!pip3 install --upgrade -q gspread

In [0]:
# Importação de dependências
import pandas as pd
from twython import Twython
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [0]:
# Solicitação de autorização para usar o Google Colab
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [0]:
# Credenciais
APP_KEY = 'B6cZTTHoeMr96AFfOsbVM2lGG'
APP_SECRET = 'QhGt5p4TsGVK8tOds1fWc7hEDIFfrgnTW94fenpl1WneWLWqfz'
OAUTH_TOKEN = '167813147-tP4vOpAZFlcfRLlqXWMfRqcBW3yvjNpzRAborOUu'
OAUTH_TOKEN_SECRET = '8UshaZNMspgdN2kRuSlkPbBJfYdu3UDzOzZt0R1F8JoU7'

In [0]:
# Conexão com o twitter
twitter = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

In [0]:
# Query no twitter
def queryTwitterToDF(q):
  query = { 'q': q, 'count': 25, 'lang': 'pt', 'result_type': 'mixed'}
  result = twitter.search(**query)
  dict_ = {'user': [],'text': [], 'favorite_count': []}  
  for status in result['statuses']:  
      dict_['user'].append(status['user'])
      dict_['text'].append(status['text'])
      dict_['favorite_count'].append(status['favorite_count'])
  df = pd.DataFrame(dict_)  
  df.sort_values(by='favorite_count')
  return df

In [0]:
# Palavras chave de direita
df_direita = pd.concat([queryTwitterToDF('#alckmin'), queryTwitterToDF('#extremaesquerda'),
                      queryTwitterToDF('#conservador'), queryTwitterToDF('#candidatodedireita'),
                      queryTwitterToDF('#bolsomito'), queryTwitterToDF('#partidodedireita'),
                      queryTwitterToDF('#PSDB'), queryTwitterToDF('#PFL'), queryTwitterToDF('#NOVO'),
                      queryTwitterToDF('#petralhas')], ignore_index=True)
df_dir = df_direita.iloc[0:200,:]

In [0]:
# Palavras chave de esquerda
df_esquerda = pd.concat([queryTwitterToDF('#lula'), queryTwitterToDF('#extremadireita'),
                      queryTwitterToDF('#lulalivre'), queryTwitterToDF('#candidatodeesquerda'),
                      queryTwitterToDF('#haddad'), queryTwitterToDF('#partidodeesquerda'),
                      queryTwitterToDF('#PT'), queryTwitterToDF('#golpe'),
                      queryTwitterToDF('#coxinhas')], ignore_index=True)
df_esq = df_esquerda.iloc[0:200,:]

In [0]:
# Palavras chave neutras
df_neutro = pd.concat([queryTwitterToDF('-#lula'), queryTwitterToDF('-#extremadireita'),
                      queryTwitterToDF('-#lulalivre'), queryTwitterToDF('-#candidatodeesquerda'),
                      queryTwitterToDF('-#haddad'), queryTwitterToDF('-#partidodeesquerda'),
                      queryTwitterToDF('-#PT'), queryTwitterToDF('-#golpe'),
                      queryTwitterToDF('-#coxinhas'), queryTwitterToDF('-#alckmin'), 
                      queryTwitterToDF('-#extremaesquerda'), queryTwitterToDF('-#conservador'), 
                      queryTwitterToDF('-#candidatodedireita'), queryTwitterToDF('-#bolsomito'), 
                      queryTwitterToDF('-#partidodedireita'), queryTwitterToDF('-#PSDB'), queryTwitterToDF('-#PFL'),
                      queryTwitterToDF('-#petralhas')], ignore_index=True)

df_n = df_neutro.iloc[0:100,:]

In [0]:
# Inserção de twittes na spreadsheet 'twitter_politico'
sh = gc.open('twitter_politico')
direita_sheet = sh.get_worksheet(0)
esquerda_sheet = sh.get_worksheet(1)
neutro_sheet = sh.get_worksheet(2)

for index, row in df_dir.iterrows():
  direita_sheet.update_acell('A' + str(index + 1), row['text'])

for index, row in df_esq.iterrows():
  esquerda_sheet.update_acell('A' + str(index + 1), row['text'])

for index, row in df_n.iterrows():
  neutro_sheet.update_acell('A' + str(index + 1), row['text'])

In [0]:
# Carregamento de dados para DataFrame.
sh = gc.open('twitter_politico')
direita_sheet = sh.get_worksheet(0)
esquerda_sheet = sh.get_worksheet(1)
neutro_sheet = sh.get_worksheet(2)

# Empilha todos os valores em uma lista
dict_ = {'text': [], 'label': []}
for row in direita_sheet.get_all_values():
  dict_['text'].append(row[0])
  dict_['label'].append(0)

for row in esquerda_sheet.get_all_values():
  dict_['text'].append(row[0])
  dict_['label'].append(1)

for row in neutro_sheet.get_all_values():
  dict_['text'].append(row[0])
  dict_['label'].append(2)
  
df = pd.DataFrame(dict_)

In [0]:
# Criando a matrix TDIDF 
count_vect = CountVectorizer()

X_TF = count_vect.fit_transform(df['text'])

tfidf_transformer = TfidfTransformer()
X = tfidf_transformer.fit_transform(X_TF)
#print(type(X))

In [0]:
# Treinamento por Aprendizagem de Máquina
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_TF, df['label'])

In [14]:
# Selecão dos twittes de um usuário
import tweepy
from tweepy import OAuthHandler
from tweepy import API

screen_name = '@alanprando'

auth = tweepy.OAuthHandler(APP_KEY, APP_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
api = tweepy.API(auth)


number_of_tweets =10
tweets = api.user_timeline(screen_name, count=10)

tmp = []

tweets_for_csv = [tweet.text for tweet in tweets]
for j in tweets_for_csv:
  tmp.append(j)

print(tmp) 

['RT @TheNoiteVideos: Acelera São Paulo https://t.co/z3waJJIIqA', 'RT @OGloboPolitica: Ex-assessor de @FlavioBolsonaro fez saques em 14 bairros, aponta relatório do Coaf https://t.co/R8UD8RIlyn https://t.co…', 'RT @joaoamoedonovo: Concordo com a avaliação de Paulo Guedes. O Sistema S deve ser revisto e suas contribuições, que atualmente são obrigat…', 'RT @joaoamoedonovo: Isso é NOVO.\n\n✅Redução de 21 para 11 secretarias\n✅Não morar em palácio\n✅Dispensar a frotas de de helicópteros e aviões…', 'RT @da_cia: Luciano Huck, meu, puta cara antenado com a Educação, bicho. Precisamos dessa galera https://t.co/twrT7bfKLJ', 'RT @gabrielazevedo: Uma informação básica: é possível se opor ao PT e ao PSL ao mesmo tempo? É possível. No Brasil, uma pessoa pode não gos…', 'RT @JovemPanNews: "Você quer ser preso?", diz Lewandowski a passageiro de avião em vídeo que circula nas redes sociais. https://t.co/teuzlb…', 'RT @janainalimasp: O NOVO assumiu o compromisso de acabar com os privilégios na políti

In [15]:
# Predição da classificação do usuário na categoria política
# Classificação: 0-direita, 1-esquerda e 2-neutro
docs_new_N= tmp
X_new_counts_N = count_vect.transform(docs_new_N)
X_new_tfidf_N = tfidf_transformer.transform(X_new_counts_N)

predicted_N = clf.predict(X_new_tfidf_N)
print(predicted_N)

for doc, category in zip(docs_new_N, predicted_N):
  if category == 0:
    print('%r => %s' % (doc, "direita"))
  elif category == 1:
    print('%r => %s' % (doc, "esquerda"))
  else:
    print('%r => %s' % (doc, "neutro"))


[1 1 0 1 0 1 1 0 1 1]
'RT @TheNoiteVideos: Acelera São Paulo https://t.co/z3waJJIIqA' => esquerda
'RT @OGloboPolitica: Ex-assessor de @FlavioBolsonaro fez saques em 14 bairros, aponta relatório do Coaf https://t.co/R8UD8RIlyn https://t.co…' => esquerda
'RT @joaoamoedonovo: Concordo com a avaliação de Paulo Guedes. O Sistema S deve ser revisto e suas contribuições, que atualmente são obrigat…' => direita
'RT @joaoamoedonovo: Isso é NOVO.\n\n✅Redução de 21 para 11 secretarias\n✅Não morar em palácio\n✅Dispensar a frotas de de helicópteros e aviões…' => esquerda
'RT @da_cia: Luciano Huck, meu, puta cara antenado com a Educação, bicho. Precisamos dessa galera https://t.co/twrT7bfKLJ' => direita
'RT @gabrielazevedo: Uma informação básica: é possível se opor ao PT e ao PSL ao mesmo tempo? É possível. No Brasil, uma pessoa pode não gos…' => esquerda
'RT @JovemPanNews: "Você quer ser preso?", diz Lewandowski a passageiro de avião em vídeo que circula nas redes sociais. https://t.co/teuzlb…' => 

In [16]:
# Importaçao de bibliotecas estatísticas
import numpy as np
import scipy.stats
import statistics

#predicted_N[1]

temp = [predicted_N[0],
        predicted_N[1],
        predicted_N[2],
        predicted_N[3],
        predicted_N[4],
        predicted_N[5],
        predicted_N[6],
        predicted_N[7],
        predicted_N[8],
        predicted_N[9]]


#print('media:', np.mean(temp))
#print('mediana:', np.median(temp))
#print('max:', np.max(temp))
#print('min:', np.min(temp))
#print('contador:', len(temp))
#print('moda:', statistics.mode(temp))

# A resposta final sobre a categoria do usuário analisado
if statistics.mode(temp) == 0:
  print('Provavelmente o usuario é de direita')
elif statistics.mode(temp) == 1:
  print('Provavelmente o usuario é de esquerda')
else:
  print('Provavelmente o usuario é neutro')


Provavelmente o usuario é de esquerda
